In [37]:
import pandas as pd
df = pd.concat([
    pd.read_parquet("complete_antipov_in_antipov_channel_until_00001.parquet"),
    pd.read_parquet("complete_antipov_in_antipov_channel2_until_00001.parquet"),
    pd.read_parquet("complete_antipov_in_antipov_channel3_until_00205.parquet"),
], axis=0)

In [50]:
posts_with_comments = df[df["Comments List"] != "[]"]

In [120]:
posts_with_comments.iloc[0]

Type                                                          text
Group                                                      antipov
Author ID                                           -1001513723099
Content          **Вот и подходит к концу 2024 год ****🎄**\n\nД...
Date                                           2024-12-31 13:34:53
Message ID                                                     289
Author                                                        None
Views                                                       223303
Reactions            ❤ 4883 👍 4351 🎄 4127 🍾 4084 🔥 4022 😁 38 🤯 35 
Shares                                                         189
Media                                                        False
Url                                       https://t.me/antipov/289
Comments List    [{"Type": "comment", "Comment Group": "antipov...
Name: 0, dtype: object

In [119]:
import json
json.loads(posts_with_comments["Comments List"].iloc[0])

[{'Type': 'comment',
  'Comment Group': 'antipov',
  'Comment Author ID': 1676045,
  'Comment Content': '',
  'Comment Date': '2025-01-08 09:54:23',
  'Comment Message ID': 1887448,
  'Comment Author': None,
  'Comment Views': None,
  'Comment Reactions': '',
  'Comment Shares': None,
  'Comment Media': 'True',
  'Comment Url': 'https://t.me/antipov/289?comment=1887448',
  'Comment Reply To Message ID': 1887443},
 {'Type': 'comment',
  'Comment Group': 'antipov',
  'Comment Author ID': 5302444135,
  'Comment Content': 'какой ?',
  'Comment Date': '2025-01-08 09:54:03',
  'Comment Message ID': 1887447,
  'Comment Author': None,
  'Comment Views': None,
  'Comment Reactions': '',
  'Comment Shares': None,
  'Comment Media': 'False',
  'Comment Url': 'https://t.me/antipov/289?comment=1887447',
  'Comment Reply To Message ID': 1887443},
 {'Type': 'comment',
  'Comment Group': 'antipov',
  'Comment Author ID': 7693825469,
  'Comment Content': 'глаз',
  'Comment Date': '2025-01-08 09:41:58',

In [123]:
import json


# construct id_to_comment
id_to_comment = {}
for i, post in posts_with_comments.iterrows():
    comments = json.loads(post["Comments List"])
    for comment in comments:
        id_to_comment[comment["Comment Message ID"]] = comment

# construct chats
target_author_id = 281485304
chats = []
for i, post in posts_with_comments.iterrows():
    comments = json.loads(post["Comments List"])
    comments_in_reverse_chronological_order = sorted(comments, key=lambda x: x["Comment Date"], reverse=True)
    target_author_comments = [
        comment for comment in comments_in_reverse_chronological_order 
        if (
            comment["Comment Author ID"] == target_author_id 
            # and
            # not (comment["Comment Reply To Message ID"] is None)
        )
    ]

    # heuristic: post id is the first comment message id
    post_id = comments[0]["Comment Reply To Message ID"]

    # post chats that have target author messages
    post_chats = []
    for comment in target_author_comments:
        chat = []
        chat.append(comment)
        while (
            comment["Comment Reply To Message ID"] is not None
        ):
            if (
                comment["Comment Reply To Message ID"] not in id_to_comment and
                comment["Comment Reply To Message ID"] != post_id
            ):
                # must be deleted message, so we drop this chat
                chat = []
                break
            elif comment["Comment Reply To Message ID"] == post_id:
                break
            else:
                comment = id_to_comment[comment["Comment Reply To Message ID"]]
                chat.append(comment)
        
        if len(chat) == 0:
            continue
        
        # only keep initiator and target author messages, imitating 2-person chat
        initiator_id = comment["Comment Author ID"]
        chat = [
            message for message in chat
            if (
                message["Comment Author ID"] == target_author_id or 
                message["Comment Author ID"] == initiator_id
            )
        ]

        chat = sorted(chat, key=lambda x: x["Comment Date"])
        chat = {
            "chat_id": ",".join([str(comment["Comment Message ID"]) for comment in chat]),
            "messages": chat
        }
        post_chats.append(chat)
    
    # filter post chats to be unique
    unique_ids = []
    for chat in sorted(
        post_chats, 
        key=lambda x: len(x["chat_id"].split(",")), 
        reverse=True
    ):
        if not any([chat["chat_id"] in id for id in unique_ids]):
            unique_ids.append(chat["chat_id"])
    print(f"Post: {post['Message ID']}; Target author messages: {len(target_author_comments)}; Unique chats: {len(unique_ids)}")
    chats.extend(post_chats)

Post: 289; Target author messages: 73; Unique chats: 0
Post: 287; Target author messages: 758; Unique chats: 0
Post: 286; Target author messages: 599; Unique chats: 0
Post: 284; Target author messages: 1; Unique chats: 0
Post: 281; Target author messages: 129; Unique chats: 60
Post: 280; Target author messages: 285; Unique chats: 0
Post: 278; Target author messages: 277; Unique chats: 0
Post: 277; Target author messages: 198; Unique chats: 0
Post: 276; Target author messages: 5; Unique chats: 2
Post: 274; Target author messages: 187; Unique chats: 0
Post: 271; Target author messages: 13; Unique chats: 0
Post: 270; Target author messages: 89; Unique chats: 0
Post: 269; Target author messages: 22; Unique chats: 17
Post: 267; Target author messages: 62; Unique chats: 0
Post: 266; Target author messages: 122; Unique chats: 60
Post: 263; Target author messages: 37; Unique chats: 0
Post: 262; Target author messages: 15; Unique chats: 0
Post: 259; Target author messages: 24; Unique chats: 10


In [116]:
chats[5]

IndexError: list index out of range

In [64]:
len(id_to_comment)

73537

In [56]:
len(target_author_comments)

73